# Моделирование восстановления золота из руды

**Описание проекта**

Подготовьте прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.

Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В вашем распоряжении данные с параметрами добычи и очистки.

Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

План работы:

1. Загрузите и подготовьте данные.
2. Исследуйте баланс классов, обучите модель без учёта дисбаланса.
3. Улучшите качество модели, учитывая дисбаланс классов. Обучите разные модели и найдите лучшую.
4. Проведите финальное тестирование.

**Описание данных**

Данные находятся в трёх файлах:

1. `gold_industry_train.csv` — обучающая выборка;
2. `gold_industry_test.csv` — тестовая выборка;
3. `gold_industry_full.csv` — исходные данные.

Данные индексируются датой и временем получения информации (признак date). Соседние по времени параметры часто похожи.
Некоторые параметры недоступны, потому что замеряются и/или рассчитываются значительно позже. Из-за этого в тестовой выборке отсутствуют некоторые признаки, которые могут быть в обучающей. Также в тестовом наборе нет целевых признаков.

Признаки технологического процесса:

1. `Rougher feed` — исходное сырье
2. `Rougher additions (или reagent additions)` — флотационные реагенты: Xanthate, Sulphate, Depressant
3. `Xanthate` — ксантогенат (промотер, или активатор флотации);
4. `Sulphate` — сульфат (на данном производстве сульфид натрия);
5. `Depressant` — депрессант (силикат натрия).
6. `Rougher process` (англ. «грубый процесс») — флотация
7. `Rougher tails` — отвальные хвосты
8. `Float banks` — флотационная установка
9. `Cleaner process` — очистка
10. `Rougher Au` — черновой концентрат золота
11. `Final Au` — финальный концентрат золота

Параметры этапов:

1. `air amount` — объём воздуха
2. `fluid levels` — уровень жидкости
3. `feed size` — размер гранул сырья
4. `feed rate` — скорость подачи

Наименование признаков по формуле [этап].[тип_параметра].[название_параметра] (`rougher.input.feed_ag`):

Возможные значения для блока [этап]:

1. `rougher` — флотация
2. `primary_cleaner` — первичная очистка
3. `secondary_cleaner` — вторичная очистка
4. `final` — финальные характеристики

Возможные значения для блока [тип_параметра]:

1. `input` — параметры сырья
2. `output` — параметры продукта
3. `state` — параметры, характеризующие текущее состояние этапа
4. `calculation` — расчётные характеристики

In [1]:
import pandas as pd
import optuna
import plotly.graph_objects as go

from caseconverter import snakecase
from ydata_profiling import ProfileReport
from IPython.display import display

from fast_ml import eda

from sklearn.model_selection import train_test_split
from sklearn.utils import resample

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, auc, roc_curve, roc_auc_score
)

In [2]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [3]:
try:
    raw_users = pd.read_csv('Churn.csv')
except:
    raw_users = pd.read_csv('/datasets/Churn.csv')